In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from deap import base, creator, tools, algorithms

In [2]:
SEED = 42
np.random.seed(SEED)

data = pd.read_csv('DATA_DESA.csv')

df = pd.DataFrame(data)

# Normalisasi MinMax
scaler = MinMaxScaler()
df['jumlah_kecelakaan'] = scaler.fit_transform(df[['jumlah_kecelakaan']])

# Menambahkan kolom lag (pergeseran)
df['jumlah_kecelakaan_lag'] = df['jumlah_kecelakaan'].shift(1)
df = df.dropna() # Menghapus baris dengan nilai NaN

In [3]:
# Memisahkan data latih dan data uji sesuai rasio (0.8 untuk 80:20)
train_size = int(0.8 * len(df))
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]

# Memisahkan fitur dan target untuk data latih
X_train = train_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_train = train_data['jumlah_kecelakaan'].values

# Memisahkan fitur dan target untuk data uji
X_test = test_data['jumlah_kecelakaan_lag'].values.reshape(-1, 1)
y_test = test_data['jumlah_kecelakaan'].values

In [4]:
# Define evaluation function
def evaluate(individual):
    C, epsilon, gamma = individual
    if C <= 0 or epsilon <= 0 or gamma <= 0:
        return float('inf'),
    svr = SVR(C=C, epsilon=epsilon, gamma=gamma)
    svr.fit(X_train, y_train)
    y_pred = svr.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    return mape,
    
# Setup DEAP framework
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Parameter ranges
parameter_ranges = [(0.01, 1), (0.001, 5), (0.001, 0.1)]

# Register genetic operators
toolbox.register("attr_float", random.uniform)
toolbox.register("individual", tools.initCycle, creator.Individual, 
                 (lambda: random.uniform(*parameter_ranges[0]),
                  lambda: random.uniform(*parameter_ranges[1]),
                  lambda: random.uniform(*parameter_ranges[2])), n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxUniform, indpb=0.1)  # Menggunakan crossover uniform
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)  # Menggunakan mutasi flipbit
toolbox.register("select", tools.selTournament, tournsize=3) # Menggunakan seleksi turnamen
toolbox.register("evaluate", evaluate)

In [5]:
# Initialize population
population_size = 50
population = toolbox.population(n=population_size)

# Genetic algorithm parameters
crossover_rate = 0.1
mutation_rate = 0.9
num_generations = 125

# Run genetic algorithm
for generation in range(num_generations):
    print(f"Generasi {generation + 1}:")
    offspring = algorithms.varAnd(population, toolbox, cxpb=crossover_rate, mutpb=mutation_rate)
    fits = toolbox.map(toolbox.evaluate, offspring)
    for fit, ind in zip(fits, offspring):
        ind.fitness.values = fit
        print(f"Individu: {ind}, MAPE: {fit[0]}")
    
    population = toolbox.select(offspring, k=len(population))

# Get the best individual
best_individual = tools.selBest(population, k=1)[0]
best_mape = best_individual.fitness.values[0]
C, epsilon, gamma = best_individual

# Test the best SVR model
best_svr = SVR(kernel='rbf', C=C, epsilon=epsilon, gamma=gamma)
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_test)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100

print("\nParameter terbaik:")
print(f"C: {C}")
print(f"Epsilon: {epsilon}")
print(f"Gamma: {gamma}")
print(f"MAPE terbaik: {best_mape}")

print("\nHasil peramalan:")
print("Prediksi:", y_pred)
print("MAPE:", mape)

Generasi 1:
Individu: [0.12296396458034745, 1.6875381619244196, 0.02716179682658401], MAPE: 57.69795835972307
Individu: [0.5912928351915909, 0.6926205041593249, 0.03830607769402039], MAPE: 57.69795835972307
Individu: [0.30058607557444317, 0.0, 0.034199800308141], MAPE: inf
Individu: [0.8720267587319569, 1.9284318236418387, 0.06776986433139005], MAPE: 57.69795835972307
Individu: [0.9094446730606272, 0.1255451158075267, 0.06196402576863924], MAPE: 58.205062436109756
Individu: [0.21396937626070767, 2.0484437531946784, 0.023525723293902314], MAPE: 57.69795835972307
Individu: [0.713764463099216, 4.513482941992785, 0.024177840710482193], MAPE: 57.69795835972307
Individu: [0.10134462836399807, 0.2725050555637749, 0.010579960649075165], MAPE: 60.38417828296818
Individu: [0.8986812452517202, 1.2449101111548175, 0.05289783940673775], MAPE: 57.69795835972307
Individu: [0.5157360221552442, 4.1843797074685085, 0.06634298016009205], MAPE: 57.69795835972307
Individu: [0.567737593684367, 1.56857714086

In [6]:
# Nilai minimum dan maksimum asli dari data yang digunakan untuk normalisasi
data_min = 0  # Sesuaikan dengan nilai minimum data asli Anda
data_max = 20  # Sesuaikan dengan nilai maksimum data asli Anda

# Denormalisasi secara manual
y_pred_denormalized = y_pred * (data_max - data_min) + data_min

#pred_denormalized =  MinMaxScaler.inverse_transform(y_pred.reshape(1, -1))
print(y_pred_denormalized)

[10.91772507 11.05455627  9.79479974  8.94290897  9.08425189 10.91772507
  9.51026354 10.64105358 10.07870742  8.66188395 10.91772507 10.2201447 ]
